In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["DRJIT_LIBLLVM_PATH"]="/opt/homebrew/opt/llvm@16/lib/libLLVM.dylib"

In [ ]:
from utils import generate_signal_map,map_to_probability

In [ ]:
percentage = 5e-2

In [ ]:
coordinates = (152,264)

In [ ]:
resolution = [480,320] # increase for higher quality of renderings

import sionna
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern

# random seed fixed for repeatability
tf.random.set_seed(1)

In [ ]:
import seaborn as sns

In [ ]:
# Load integrated scene
scene = load_scene("../linkhall1cm/linkhall1cm.xml") 
#scene = load_scene(sionna.rt.scene.munich)


In [ ]:
# define TX and RX antennas and RF information

# Set the scattering coefficient of the radio material in the scene
# to a non-zero value to enable scattering

for key,mat in scene.radio_materials.items():
    print(mat,key, mat.well_defined)
    mat.scattering_coefficient=0

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=8,
                             num_cols=8,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V",
                             polarization_model=2)

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="V",
                             polarization_model=2)

scene.frequency = 3.5e9 # set the carrier frequency, 3.5 GHz is the default
cm_cell_size = np.array([1,1]) # Each cell is 0.5mx0.5m 

In [ ]:
# transmitters = [
#     {"position":[5,0,136], "orientation":[0,0,0], "lookat":[30,-27,0]},
#     {"position":[5,-30,136], "orientation":[0,0,0], "lookat":[30,27,0]},
#     {"position":[8,-70,130], "orientation":[0,0,0], "lookat":[30,-70,0]},
#     {"position":[-58,55,136], "orientation":[0,0,0], "lookat":[0,55,0]}
# ]

# receivers = [
#     {"position":[30,-27,0]},
#     {"position":[30,27,0]},
#     {"position":[30,-70,0]},
#     {"position":[0,55,0]},
# ]

# transmitters = [
#     {"position":[0, 20, 22], "orientation":[0,0,0], "lookat":[0,35,5]},
# ]

# receivers = [
#     {"position":[0,35,5]},
# ]

In [ ]:
# point_measuraments = []
# maps = []
# probs = []
# cms = []
# for i in range(len(transmitters[:1])):
#     print("starting map n: ",i)
#     map, cm = generate_signal_map(scene, cm_cell_size, transmitter = transmitters[i], preview=False,height=None, receiver=receivers[i],orientation=(0,0,0))
#     maps.append(map)
#     point = map[coordinates[0]][coordinates[1]]
#     point_measuraments.append(point)
#     #prob = map_to_probability(point,map,percentage)
#     #probs.append(prob)
#     cms.append(cm)
#     # sns.heatmap(map,vmin=-200,vmax=-20)
#     # plt.show()

In [ ]:
scene.remove("tx")
scene.remove("rx")
transmitter = {"position":[3, 9, 5], "orientation":[0,0,0], "lookat":[16,16,2]}
tx = Transmitter("tx", position=transmitter["position"],
                    orientation=[0,0,0])
receiver = {"position":[16,16,2]}
rx = Receiver("rx", position=receiver["position"], orientation=[0,0,0])
scene.add(rx)
scene.add(tx)
tx.look_at(rx)

In [ ]:
custom_material = RadioMaterial("my_material_2",
                                relative_permittivity=4.0,
                                conductivity=0.1,
                                scattering_coefficient=0,
                                xpd_coefficient=0.1,
                                scattering_pattern=LambertianPattern())

In [ ]:
for key, val in scene.objects.items():
    obj = scene.get(key)
    obj.radio_material = custom_material

In [ ]:
paths = scene.compute_paths()

In [ ]:
#scene.preview(paths=paths)

In [ ]:
coverage_map = scene.coverage_map(cm_cell_size=[1,1], # Configure size of each cell
                        num_samples=1e6, diffraction=False, scattering=False,
                        cm_center=(0,0,0.3),
                        cm_size=(200,200),
                        cm_orientation=(0,0,0))

In [ ]:
scene.preview(coverage_map=coverage_map, paths=paths)

In [ ]:
for prob in probs:
    print(prob[coordinates[0]][coordinates[1]], np.max(prob))

In [ ]:
f, (ax1, ax2,ax3,ax5) = plt.subplots(1, 4,figsize=(20, 5))
f.suptitle("Probability for single antennas")
sns.heatmap(probs[0], ax=ax1, cmap="viridis")
sns.heatmap(probs[1], ax=ax2,cmap="viridis")
sns.heatmap(probs[2], ax=ax3, cmap="viridis")
sns.heatmap(probs[3], ax=ax5, cmap="viridis")
plt.show()

In [ ]:
prob_final = 1
for prob in probs:
    prob_final *= prob

prob_final[coordinates[0]][coordinates[1]]

In [ ]:
np.max(prob_final)

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
sns.heatmap(prob_final,
            #norm=LogNorm()
            )
plt.title("Location probability")
plt.axvline(coordinates[1],linewidth=0.7)
plt.axhline(coordinates[0],linewidth=0.7, label="real location")
plt.legend()
plt.show()

# Second timestep

In [ ]:
coordinates_2 = coordinates

In [ ]:
point_measuraments_2 = []
maps_2 = []
probs_2 = []
cms_2 = []
for i in range(len(transmitters)):
    print("starting map n: ",i)
    map, cm = generate_signal_map(scene, cm_cell_size, transmitter = transmitters[i], preview=False,height=None, receiver=receivers[i],orientation=(0,0,0))
    maps_2.append(map)
    point = map[coordinates_2[0]][coordinates_2[1]]
    point_measuraments_2.append(point)
    prob = map_to_probability(point,map,percentage)
    probs_2.append(prob)
    cms_2.append(cm)
    sns.heatmap(map,vmin=-200,vmax=-20)
    plt.show()

In [ ]:
prob_final_2 = 1
for prob in probs_2:
    prob_final_2 *= prob

prob_final_2[coordinates_2[0]][coordinates_2[1]]

In [ ]:
print(np.max(prob_final_2))

In [ ]:
sns.heatmap(prob_final_2,
            #norm=LogNorm()
            )
plt.title("Location probability")
plt.axvline(coordinates_2[1],linewidth=0.7)
plt.axhline(coordinates_2[0],linewidth=0.7, label="real location")
plt.legend()
plt.show()

In [ ]:
sns.heatmap(prob_final * prob_final_2,
            #norm=LogNorm()
            )

# 3rd

In [ ]:
coordinates_3 = coordinates
point_measuraments_3 = []
maps_3 = []
probs_3 = []
cms_3 = []
for i in range(len(transmitters)):
    print("starting map n: ",i)
    map, cm = generate_signal_map(scene, cm_cell_size, transmitter = transmitters[i], preview=False,height=None, receiver=receivers[i],orientation=(0,0,0))
    maps_3.append(map)
    point = map[coordinates_3[0]][coordinates_3[1]]
    point_measuraments_3.append(point)
    prob = map_to_probability(point,map,percentage)
    probs_3.append(prob)
    cms_3.append(cm)
    sns.heatmap(map,vmin=-200,vmax=-20)
    plt.show()

In [ ]:
prob_final_3 = 1
for prob in probs_3:
    prob_final_3 *= prob

prob_final_3[coordinates_3[0]][coordinates_3[1]]

In [ ]:
print(np.max(prob_final_3))

In [ ]:
sns.heatmap(prob_final_3,
            #norm=LogNorm()
            )
plt.title("Location probability")
plt.axvline(coordinates_3[1],linewidth=0.7)
plt.axhline(coordinates_3[0],linewidth=0.7, label="real location")
plt.legend()
plt.show()

In [ ]:
sns.heatmap(prob_final * prob_final_2 * prob_final_3,
            #norm=LogNorm()
            )
plt.axvline(coordinates_3[1],linewidth=0.7)
plt.axhline(coordinates_3[0],linewidth=0.7, label="real location")
plt.legend()
plt.show()

In [ ]:
coordinates_4 = coordinates
point_measuraments_4 = []
maps_4 = []
probs_4 = []
cms_4 = []
for i in range(len(transmitters)):
    print("starting map n: ",i)
    map, cm = generate_signal_map(scene, cm_cell_size, transmitter = transmitters[i], preview=False,height=None, receiver=receivers[i],orientation=(0,0,0))
    maps_4.append(map)
    point = map[coordinates_4[0]][coordinates_4[1]]
    point_measuraments_4.append(point)
    prob = map_to_probability(point,map,percentage)
    probs_4.append(prob)
    cms_4.append(cm)
    sns.heatmap(map,vmin=-200,vmax=-20)
    plt.show()

In [ ]:
prob_final_4 = 1
for prob in probs_4:
    prob_final_4 *= prob

prob_final_4[coordinates_4[0]][coordinates_4[1]]

In [ ]:


print((prob_final * prob_final_2 * prob_final_3)[coordinates[0]][coordinates[1]],max3)

In [ ]:
max4 = np.max(prob_final * prob_final_2 * prob_final_3 * prob_final_4)
max3 = np.max(prob_final * prob_final_2 * prob_final_3)
max2 = np.max(prob_final * prob_final_2)
max1 = np.max(prob_final)
f, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(20, 5))
ax1.set_title("timestep 1")
ax2.set_title("timestep 2")
ax3.set_title("timestep 3")
ax4.set_title("timestep 4")
f.suptitle("Location probability")
sns.heatmap(prob_final/max1, ax=ax1,cmap="viridis")
sns.heatmap(prob_final * prob_final_2/max2, ax=ax2, cmap="viridis")
sns.heatmap(prob_final * prob_final_2 * prob_final_3/max3, ax=ax3, cmap="viridis")
#sns.heatmap((prob_final * prob_final_2 * prob_final_3 * prob_final_4)/max4, ax=ax4, cmap="viridis")
plt.axvline(coordinates_3[1],linewidth=0.7)
plt.axhline(coordinates_3[0],linewidth=0.7, label="real location")
plt.legend()
plt.show()

In [ ]:
from PIL import Image 
  
# open method used to open different extension image file 
im = Image.open(r"TOP_415810.tif")  
  
# This method will show image in any image viewer  
im.show()  

In [ ]:
from pyproj import Proj, transform
import geopandas as gpd
import shapely
 
shapefile = gpd.read_file("footprints/Projektflaeche_Footprints_NORD.shp")
 
source_proj = Proj(init='epsg:32632')  # UTM Zone 32N
target_proj = Proj(init='epsg:4326')   # WGS84 (latitude and longitude)
 
# Transform coordinates
x, y = shapely.get_coordinates(shapefile["geometry"][0])[0]
lon, lat = transform(source_proj, target_proj, x, y)
 
print("Longitude:", lon)
print("Latitude:", lat)
shapefile

In [ ]:
shapefile